In [994]:
# !pip install -q pandas
# !pip install -q requests google-auth google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [995]:
import copy
import json
import pandas as pd
import requests
import re

In [996]:
default_headers = {
    # 'Accept': '*/*;q=0.5, text/javascript, application/javascript, application/ecmascript, application/x-ecmascript' ,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en-US,en;q=0.9,fa;q=0.8' ,
    'Connection': 'keep-alive' ,
    # 'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8' ,
    'Content-Type': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7' ,
    'Origin': 'https://ais.usvisa-info.com' ,
    'Referer': 'https://ais.usvisa-info.com/en-am/niv/users/sign_in' ,
    'Sec-Fetch-Dest': 'empty' ,
    'Sec-Fetch-Mode': 'cors' ,
    'Sec-Fetch-Site': 'same-origin' ,
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36' ,
    # 'X-CSRF-Token': '2eCWn/ixpL3flKpJVAGH+Er3KuGVDCSyCNmvqdInBEvXkiigk0txMOnufLYfBbI6RRU9K3xTUOTVgg42oRL5tA==' ,
    'X-Requested-With': 'XMLHttpRequest' ,
    'sec-ch-ua': '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"' ,
    'sec-ch-ua-mobile': '?0' ,
    'sec-ch-ua-platform': '"macOS"' ,
  }

In [997]:
res = requests.get(
    'https://ais.usvisa-info.com/en-ke/niv/users/sign_in',
    headers = default_headers)
res.status_code

200

In [998]:
cookies = res.cookies
csrf_token = None
csrf_match = re.search(r'<meta name="csrf-token" content="(.*?)"',
                      str(res.content))
if csrf_match:
  csrf_token = csrf_match.group(1)
  print(f'X-CSRF-Token retrieved.')
else:
  print("No X-CSRF-Token found.")

X-CSRF-Token retrieved.


In [999]:
headers = copy.deepcopy(default_headers)
headers['X-CSRF-Token'] = csrf_token
headers['Content-Type'] = 'application/x-www-form-urlencoded; charset=UTF-8'
res = requests.post('https://ais.usvisa-info.com/en-ke/niv/users/sign_in',
    headers=headers,
    cookies=cookies,
    data={
       'user[email]': 'delaramali.delaramali@gmail.com',
       'user[password]': 'Kenya@2024',
       'policy_confirmed': '1',
       'commit': 'Sign In',
    })
print(f'status code: {res.status_code}')
print(f'{res.content.decode("utf-8")}')

status code: 200
$('#authFactor.reveal').foundation('close')
$.blockUI( {message: "", css: { border: 'none', padding: '15px', backgroundColor: '#000', '-webkit-border-radius': '10px', '-moz-border-radius': '10px', opacity: .5, color: '#fff'}})
window.location.pathname="/en-ke/niv/account"



In [1000]:
res = requests.get(
    'https://ais.usvisa-info.com/en-ke/niv/account',
    headers = default_headers,
    cookies = res.cookies,
    allow_redirects=False)
res.status_code

302

In [1001]:
res = requests.get(
    'https://ais.usvisa-info.com/en-ke/niv/groups/43464236',
    headers = default_headers,
    cookies = res.cookies,
    allow_redirects=False)
res.status_code

200

In [1002]:
res = requests.get(
    'https://ais.usvisa-info.com/en-ke/niv/schedule/61475787/appointment?applicants%5B%5D=72890185&applicants%5B%5D=72890374&confirmed_limit_message=1&commit=Continue',
    headers = default_headers,
    cookies = res.cookies,
    allow_redirects=False)
print(f'response code: {res.status_code}')
if 'Reschedule' in str(res.content):
  print('Successfully loaded the appointment page.')
else:
  print('Failed to load the appointment page')

response code: 200
Successfully loaded the appointment page.


In [1003]:
headers = copy.deepcopy(default_headers)
headers['Accept'] = 'application/json, text/javascript, */*; q=0.01'
res = requests.get(
    'https://ais.usvisa-info.com/en-ke/niv/schedule/61475787/appointment/days/104.json?appointments[expedite]=false',
    headers = default_headers,
    cookies = res.cookies,
    allow_redirects=False)
res.status_code

200

In [1004]:
date_to_comapre = '2024-12-01'

dates_df = pd.json_normalize(json.loads(res.content))
first_avail_date = dates_df[dates_df['date'] < date_to_comapre]
if first_avail_date.empty:
  print('No early appointments available.')
else:
  first_avail_date = first_avail_date.sort_values(by='date', ascending=True).head(1).values[0][0]
  print(f'First available appointment: {first_avail_date}')

No early appointments available.


In [1005]:
import pytz
from datetime import timedelta, datetime


def create_event(service):
    # Create an event on the primary calendar
    # Get the Hawaiian timezone
    hawaiian_tz = pytz.timezone('Pacific/Honolulu')

    # Get the current time in UTC
    utc_now = datetime.now(pytz.utc)

    # Convert the UTC time to Hawaiian time
    start_time = utc_now.astimezone(hawaiian_tz) + timedelta(minutes=5)

    event = {
      'summary': 'US Visa Appointment',
      'description': f'First available appointment: {first_avail_date}',
      'start': {
        'dateTime': start_time.isoformat(),
        'timeZone': 'Pacific/Honolulu',
      },
      'end': {
        'dateTime': (start_time + timedelta(hours=1)).isoformat(),
        'timeZone': 'Pacific/Honolulu',
      },
      'creator': {
         'email': 'alireza.ali@gmail.com',
      },
      # 'recurrence': [
      #   'RRULE:FREQ=DAILY;COUNT=2'
      # ],
      'attendees': [
        {'email': 'alireza.ali@gmail.com'},
        # {'email': 'example2@gmail.com'},
      ],
      'reminders': {
        'useDefault': False,
        'overrides': [
          {'method': 'email', 'minutes': 1},
          {'method': 'popup', 'minutes': 1},
        ],
      },
    }

    event = service.events().insert(calendarId='primary', body=event).execute()
    print(f'Event created: {event.get("htmlLink")}')

In [1006]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os

# /content/drive/MyDrive/Colab Notebooks/client_secret_1020799066832-37ba3r1pimcrf4cf8l0qub4ga13u2e1t.apps.googleusercontent.com.json

SCOPES = ['https://www.googleapis.com/auth/calendar.events']

def authenticate_google_calendar():
    creds = None
    redirect_uri = "https://developers.google.com/oauthplayground"  # Specify the redirect_uri

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret_1020799066832-37ba3r1pimcrf4cf8l0qub4ga13u2e1t.apps.googleusercontent.com.json',
                SCOPES)
            flow.redirect_uri = redirect_uri
            # Removed redirect_uri from this call as it's already specified above
            auth_url, _ = flow.authorization_url(prompt='consent')

            print(f'Please go to this URL and authorize the app: {auth_url}')
            code = input('Enter the authorization code: ')
            flow.fetch_token(code=code)

            creds = flow.credentials

        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('calendar', 'v3', credentials=creds)

In [1007]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

import os
import os.path

# /content/drive/MyDrive/Colab Notebooks/client_secret_1020799066832-37ba3r1pimcrf4cf8l0qub4ga13u2e1t.apps.googleusercontent.com.json

SCOPES = ['https://www.googleapis.com/auth/calendar.events']

def authenticate_google_calendar():
    creds = None
    redirect_uri = "https://developers.google.com/oauthplayground"  # Specify the redirect_uri

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret_1020799066832-37ba3r1pimcrf4cf8l0qub4ga13u2e1t.apps.googleusercontent.com.json',
                SCOPES)
            flow.redirect_uri = redirect_uri
            # Removed redirect_uri from this call as it's already specified above
            auth_url, _ = flow.authorization_url(prompt='consent')

            print(f'Please go to this URL and authorize the app: {auth_url}')
            code = input('Enter the authorization code: ')
            flow.fetch_token(code=code)

            creds = flow.credentials

        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('calendar', 'v3', credentials=creds)

In [1008]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os

# /content/drive/MyDrive/Colab Notebooks/client_secret_1020799066832-37ba3r1pimcrf4cf8l0qub4ga13u2e1t.apps.googleusercontent.com.json

SCOPES = ['https://www.googleapis.com/auth/calendar.events']

def authenticate_google_calendar():
    creds = None
    redirect_uri = "https://developers.google.com/oauthplayground"  # Specify the redirect_uri

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secret_1020799066832-37ba3r1pimcrf4cf8l0qub4ga13u2e1t.apps.googleusercontent.com.json',
                SCOPES)
            flow.redirect_uri = redirect_uri
            # Removed redirect_uri from this call as it's already specified above
            auth_url, _ = flow.authorization_url(prompt='consent')

            print(f'Please go to this URL and authorize the app: {auth_url}')
            code = input('Enter the authorization code: ')
            flow.fetch_token(code=code)

            creds = flow.credentials

        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('calendar', 'v3', credentials=creds)

In [1009]:
print(f'{datetime.now().isoformat()}\n')
service = authenticate_google_calendar()
if not isinstance(first_avail_date,str):
  print('No early appointments available.')
else:
  create_event(service)

dates_df.to_csv('dates_df.csv', index=False)
dates_df.head(10)

2024-09-26T21:13:31.396251

No early appointments available.


,date,business_day
0,2025-02-10,True
1,2025-02-11,True
2,2025-02-12,True
3,2025-02-18,True
4,2025-02-19,True
5,2025-02-20,True
6,2025-02-24,True
7,2025-02-25,True
